In [2]:
import logging
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from src.turbofan import Turbofan
from utils.configs import (
    config_ex23,
    DEFAULT_CONFIG_TURBOFAN
)

pio.templates.default = "plotly"

### Validação 1: Exemplo 2.2

In [3]:
# turbofan = Turbofan(config_ex22)
# turbofan.set_air_flow(533)
# turbofan.print_outputs()

### Teste 1 de PRP-38

In [4]:
# turbofan = Turbofan(config_teste1)
# turbofan.set_air_flow(533)
# turbofan.print_outputs()

### CFM56-7B27

In [8]:
# --- Configuração de Logger e Diretórios ---
LOG_DIR = "logs"
IMAGES_DIR = Path("images") # Define um diretório para salvar as imagens
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)
if not os.path.exists(IMAGES_DIR): os.makedirs(IMAGES_DIR)

logger = logging.getLogger("validation_plot")
logger.setLevel(logging.INFO)
if not logger.handlers:
    file_handler = logging.FileHandler(os.path.join(LOG_DIR, "validation.log"), mode='w', encoding='utf-8')
    console_handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
lto_thrust_percentages = [0.07, 0.30, 0.85, 1.00]

# ==============================================================================
# 1. CONFIGURAÇÃO E CALIBRAÇÃO DO MOTOR
# ==============================================================================
logger.info("Iniciando calibração do motor para validação...")

# Configuração base e do motor
motor = "CFM56-7B27"
engine_config = {
    "bpr": 5.0,
    "prf": 1.5,
    "prc": 28.6 / 1.5,
}
icao_fuel_flow_kgs = [0.110, 0.343, 1.031, 1.293]
rated_thrust_kN = 121.4  # Empuxo nominal (100%)
fuel_flow_takeoff_kgs = 1.293  # Consumo em 100% de empuxo
t_04_bounds = (1400, 1800)
m_dot_bounds = (400, 800)

# Calibração
config_base = DEFAULT_CONFIG_TURBOFAN.copy()
config_base.update(engine_config)
turbofan = Turbofan(config_base)
optimization_status = turbofan.calibrate_turbofan(
    rated_thrust_kN=rated_thrust_kN,
    fuel_flow_kgs=fuel_flow_takeoff_kgs,
    t04_bounds=t_04_bounds,
    m_dot_bounds= m_dot_bounds
)
logger.info(f"Status da Calibração: {optimization_status}")

# Salva imagem da calibração
fig_calibracao = turbofan.plot_calibration_result(
    fuel_flow_takeoff_kgs/rated_thrust_kN,
    t04_range=np.arange(t_04_bounds[0], t_04_bounds[1], 1),
    title=f"Calibração do {motor}",
)
fig_calibracao.show()
fig_calibracao.write_image(IMAGES_DIR / f"{motor}_calibration.png", width=800, height=500, scale=2)

# ==============================================================================
# 2. GERAÇÃO DE DADOS (Meta-Modelo vs. ICAO)
# ==============================================================================
logger.info("Gerando dados para os pontos do ciclo LTO...")

# Calcula o consumo de combustível do seu modelo em cada ponto LTO
meta_modelo_fuel_flow_kgs = []
for thrust_perc in lto_thrust_percentages:
    turbofan.update_environment(percentage_of_rated_thrust=thrust_perc)
    meta_modelo_fuel_flow_kgs.append(turbofan.get_fuel_consumption())

# Normaliza os resultados pelo consumo de decolagem
meta_modelo_normalized = [ff / fuel_flow_takeoff_kgs for ff in meta_modelo_fuel_flow_kgs]

# Normaliza os dados da ICAO pelo consumo de decolagem
icao_normalized = [ff / fuel_flow_takeoff_kgs for ff in icao_fuel_flow_kgs]

# ==============================================================================
# 3. CRIAÇÃO DO DATAFRAME
# ==============================================================================

# Estrutura os dados para o Plotly
plot_df = pd.DataFrame({
    "Tração Líquida / Tração TO [-]": lto_thrust_percentages,
    "Meta-Modelo": meta_modelo_normalized,
    "ICAO": icao_normalized,
})
logger.info(f"DataFrame para plotagem:\n{plot_df}")

# ==============================================================================
# 4. PLOTAGEM COM PLOTLY GRAPH_OBJECTS (GO)
# ==============================================================================
logger.info("Gerando gráfico...")

# Inicializa a figura
fig = go.Figure()

# --- Adiciona a Linha da ICAO (Vermelha, Sólida) ---
fig.add_trace(go.Scatter(
    x=plot_df["Tração Líquida / Tração TO [-]"],
    y=plot_df["ICAO"],
    mode='lines+markers',  # Linhas e marcadores
    name='ICAO',
    line=dict(color='red', width=2, dash='solid'),
    marker=dict(color='red', size=8, symbol='circle')
))

# --- Adiciona a Linha do Meta-Modelo (Preta, Tracejada) ---
fig.add_trace(go.Scatter(
    x=plot_df["Tração Líquida / Tração TO [-]"],
    y=plot_df["Meta-Modelo"],
    mode='lines+markers',  # Linhas e marcadores
    name='Meta-Modelo',
    line=dict(color='black', width=2, dash='dash'),
    marker=dict(color='black', size=8, symbol='circle')
))

# --- Customiza o Layout do Gráfico ---
fig.update_layout(
    title=f"Validação do Meta-Modelo vs. ICAO (Motor: {motor})",
    xaxis_title="Tração Líquida / Tração TO [-]",
    yaxis_title="Vazão de Combustível / Vazão TO [-]",
    # Define os limites dos eixos para corresponder ao seu exemplo
    xaxis=dict(range=[0, 1.1]),
    yaxis=dict(range=[0, 1.1]),
    # Posiciona a legenda no canto inferior direito
    legend=dict(
        x=0.6, y=0.1,
        bgcolor='rgba(255, 255, 255, 0.5)', # Fundo semitransparente
        bordercolor='rgba(0, 0, 0, 0.5)',
        borderwidth=1
    )
)

# ==============================================================================
# 5. SALVAR E EXIBIR
# ==============================================================================

# Salva a imagem na pasta 'images'
file_name = f"{motor}_validation_plot.png"
fig.write_image(IMAGES_DIR / file_name, width=800, height=500, scale=2)
logger.info(f"Gráfico salvo em: {IMAGES_DIR / file_name}")

# Exibe o gráfico
fig.show()

2025-11-03 00:02:22,344 - INFO - Iniciando calibração do motor para validação...
2025-11-03 00:02:22,350 - INFO - Status da Calibração: {'success': True, 'stage_sequence': ['tsfc_then_thrust'], 'message_t04': 'Solution found.', 'message_mdot': 'Solution found.', 'optimal_t04': 1555.987, 'optimal_mass_flow_rate': 401.107, 'final_thrust_kN': 121.4, 'final_tsfc': 0.01065}
2025-11-03 00:02:22,351 - src.turbofan - INFO - Calculando TSFC para visualização no intervalo T04 de 1400K a 1799K...


2025-11-03 00:02:23,904 - INFO - Gerando dados para os pontos do ciclo LTO...
2025-11-03 00:02:23,920 - INFO - DataFrame para plotagem:
   Tração Líquida / Tração TO [-]  Meta-Modelo      ICAO
0                            0.07     0.098838  0.085073
1                            0.30     0.251768  0.265275
2                            0.85     0.787508  0.797370
3                            1.00     1.000000  1.000000
2025-11-03 00:02:23,920 - INFO - Gerando gráfico...
2025-11-03 00:02:25,347 - INFO - Gráfico salvo em: images\CFM56-7B27_validation_plot.png


### PW1133G

In [6]:
# --- Configuração de Logger e Diretórios ---
LOG_DIR = "logs"
IMAGES_DIR = Path("images") # Define um diretório para salvar as imagens
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)
if not os.path.exists(IMAGES_DIR): os.makedirs(IMAGES_DIR)

logger = logging.getLogger("validation_plot")
logger.setLevel(logging.INFO)
if not logger.handlers:
    file_handler = logging.FileHandler(os.path.join(LOG_DIR, "validation.log"), mode='w', encoding='utf-8')
    console_handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
lto_thrust_percentages = [0.07, 0.30, 0.85, 1.00]

# ==============================================================================
# 1. CONFIGURAÇÃO E CALIBRAÇÃO DO MOTOR
# ==============================================================================
logger.info("Iniciando calibração do motor para validação...")

# Configuração base e do motor
motor = "PW1133G"
engine_config = {
    "bpr": 11.6,
    "prf": 1.5,
    "prc": 38.1 / 1.5,
}
icao_fuel_flow_kgs = [0.099, 0.278, 0.839, 1.023]
rated_thrust_kN = 147.3  # Empuxo nominal (100%)
fuel_flow_takeoff_kgs = 1.023  # Consumo em 100% de empuxo
t_04_bounds = (1500, 1900)
m_dot_bounds = (400, 800)

# Calibração
config_base = DEFAULT_CONFIG_TURBOFAN.copy()
config_base.update(engine_config)
turbofan = Turbofan(config_base)
optimization_status = turbofan.calibrate_turbofan(
    rated_thrust_kN=rated_thrust_kN,
    fuel_flow_kgs=fuel_flow_takeoff_kgs,
    t04_bounds=t_04_bounds,
    m_dot_bounds= m_dot_bounds
)
logger.info(f"Status da Calibração: {optimization_status}")

# Salva imagem da calibração
fig_calibracao = turbofan.plot_calibration_result(
    fuel_flow_takeoff_kgs/rated_thrust_kN,
    t04_range=np.arange(t_04_bounds[0], t_04_bounds[1], 1),
    title=f"Calibração do {motor}",
)
fig_calibracao.show()
fig_calibracao.write_image(IMAGES_DIR / f"{motor}_calibration.png", width=800, height=500, scale=2)

# ==============================================================================
# 2. GERAÇÃO DE DADOS (Meta-Modelo vs. ICAO)
# ==============================================================================
logger.info("Gerando dados para os pontos do ciclo LTO...")

# Calcula o consumo de combustível do seu modelo em cada ponto LTO
meta_modelo_fuel_flow_kgs = []
for thrust_perc in lto_thrust_percentages:
    turbofan.update_environment(percentage_of_rated_thrust=thrust_perc)
    meta_modelo_fuel_flow_kgs.append(turbofan.get_fuel_consumption())

# Normaliza os resultados pelo consumo de decolagem
meta_modelo_normalized = [ff / fuel_flow_takeoff_kgs for ff in meta_modelo_fuel_flow_kgs]

# Normaliza os dados da ICAO pelo consumo de decolagem
icao_normalized = [ff / fuel_flow_takeoff_kgs for ff in icao_fuel_flow_kgs]

# ==============================================================================
# 3. CRIAÇÃO DO DATAFRAME
# ==============================================================================

# Estrutura os dados para o Plotly
plot_df = pd.DataFrame({
    "Tração Líquida / Tração TO [-]": lto_thrust_percentages,
    "Meta-Modelo": meta_modelo_normalized,
    "ICAO": icao_normalized,
})
logger.info(f"DataFrame para plotagem:\n{plot_df}")

# ==============================================================================
# 4. PLOTAGEM COM PLOTLY GRAPH_OBJECTS (GO)
# ==============================================================================
logger.info("Gerando gráfico...")

# Inicializa a figura
fig = go.Figure()

# --- Adiciona a Linha da ICAO (Vermelha, Sólida) ---
fig.add_trace(go.Scatter(
    x=plot_df["Tração Líquida / Tração TO [-]"],
    y=plot_df["ICAO"],
    mode='lines+markers',  # Linhas e marcadores
    name='ICAO',
    line=dict(color='red', width=2, dash='solid'),
    marker=dict(color='red', size=8, symbol='circle')
))

# --- Adiciona a Linha do Meta-Modelo (Preta, Tracejada) ---
fig.add_trace(go.Scatter(
    x=plot_df["Tração Líquida / Tração TO [-]"],
    y=plot_df["Meta-Modelo"],
    mode='lines+markers',  # Linhas e marcadores
    name='Meta-Modelo',
    line=dict(color='black', width=2, dash='dash'),
    marker=dict(color='black', size=8, symbol='circle')
))

# --- Customiza o Layout do Gráfico ---
fig.update_layout(
    title=f"Validação do Meta-Modelo vs. ICAO (Motor: {motor})",
    xaxis_title="Tração Líquida / Tração TO [-]",
    yaxis_title="Vazão de Combustível / Vazão TO [-]",
    # Define os limites dos eixos para corresponder ao seu exemplo
    xaxis=dict(range=[0, 1.1]),
    yaxis=dict(range=[0, 1.1]),
    # Posiciona a legenda no canto inferior direito
    legend=dict(
        x=0.6, y=0.1,
        bgcolor='rgba(255, 255, 255, 0.5)', # Fundo semitransparente
        bordercolor='rgba(0, 0, 0, 0.5)',
        borderwidth=1
    )
)

# ==============================================================================
# 5. SALVAR E EXIBIR
# ==============================================================================

# Salva a imagem na pasta 'images'
file_name = f"{motor}_validation_plot.png"
fig.write_image(IMAGES_DIR / file_name, width=800, height=500, scale=2)
logger.info(f"Gráfico salvo em: {IMAGES_DIR / file_name}")

# Exibe o gráfico
fig.show()

2025-11-02 21:52:45,508 - INFO - Iniciando calibração do motor para validação...
2025-11-02 21:52:45,524 - INFO - Status da Calibração: {'success': True, 'stage_sequence': ['tsfc_then_thrust'], 'message_t04': 'Solution found.', 'message_mdot': 'Solution found.', 'optimal_t04': 1706.9, 'optimal_mass_flow_rate': 599.001, 'final_thrust_kN': 147.3, 'final_tsfc': 0.00695}
2025-11-02 21:52:45,524 - src.turbofan - INFO - Calculando TSFC para visualização no intervalo T04 de 1500K a 1899K...


2025-11-02 21:52:47,000 - INFO - Gerando dados para os pontos do ciclo LTO...
2025-11-02 21:52:47,016 - INFO - DataFrame para plotagem:
   Tração Líquida / Tração TO [-]  Meta-Modelo      ICAO
0                            0.07     0.068557  0.096774
1                            0.30     0.232008  0.271750
2                            0.85     0.876446  0.820137
3                            1.00     1.000000  1.000000
2025-11-02 21:52:47,016 - INFO - Gerando gráfico...
2025-11-02 21:52:48,438 - INFO - Gráfico salvo em: images\PW1133G_validation_plot.png


### SPEY Mk511

In [7]:
# --- Configuração de Logger e Diretórios ---
LOG_DIR = "logs"
IMAGES_DIR = Path("images") # Define um diretório para salvar as imagens
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)
if not os.path.exists(IMAGES_DIR): os.makedirs(IMAGES_DIR)

logger = logging.getLogger("validation_plot")
logger.setLevel(logging.INFO)
if not logger.handlers:
    file_handler = logging.FileHandler(os.path.join(LOG_DIR, "validation.log"), mode='w', encoding='utf-8')
    console_handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
lto_thrust_percentages = [0.07, 0.30, 0.85, 1.00]

# ==============================================================================
# 1. CONFIGURAÇÃO E CALIBRAÇÃO DO MOTOR
# ==============================================================================
logger.info("Iniciando calibração do motor para validação...")

# Configuração base e do motor
motor = "SPEY Mk511"
engine_config = {
    "bpr": 0.64,
    "prf": 1.5,
    "prc": 19.9 / 1.5,
}
icao_fuel_flow_kgs = [0.12, 0.28, 0.730, 0.890]
rated_thrust_kN = 50.7  # Empuxo nominal (100%)
fuel_flow_takeoff_kgs = 0.890  # Consumo em 100% de empuxo
t_04_bounds = (1100, 1600)
m_dot_bounds = (100, 800)

# Calibração
config_base = DEFAULT_CONFIG_TURBOFAN.copy()
config_base.update(engine_config)
turbofan = Turbofan(config_base)
optimization_status = turbofan.calibrate_turbofan(
    rated_thrust_kN=rated_thrust_kN,
    fuel_flow_kgs=fuel_flow_takeoff_kgs,
    t04_bounds=t_04_bounds,
    m_dot_bounds= m_dot_bounds
)
logger.info(f"Status da Calibração: {optimization_status}")

# Salva imagem da calibração
fig_calibracao = turbofan.plot_calibration_result(
    fuel_flow_takeoff_kgs/rated_thrust_kN,
    t04_range=np.arange(t_04_bounds[0], t_04_bounds[1], 1),
    title=f"Calibração do {motor}",
)
fig_calibracao.show()
fig_calibracao.write_image(IMAGES_DIR / f"{motor}_calibration.png", width=800, height=500, scale=2)

# ==============================================================================
# 2. GERAÇÃO DE DADOS (Meta-Modelo vs. ICAO)
# ==============================================================================
logger.info("Gerando dados para os pontos do ciclo LTO...")

# Calcula o consumo de combustível do seu modelo em cada ponto LTO
meta_modelo_fuel_flow_kgs = []
for thrust_perc in lto_thrust_percentages:
    turbofan.update_environment(percentage_of_rated_thrust=thrust_perc)
    meta_modelo_fuel_flow_kgs.append(turbofan.get_fuel_consumption())

# Normaliza os resultados pelo consumo de decolagem
meta_modelo_normalized = [ff / fuel_flow_takeoff_kgs for ff in meta_modelo_fuel_flow_kgs]

# Normaliza os dados da ICAO pelo consumo de decolagem
icao_normalized = [ff / fuel_flow_takeoff_kgs for ff in icao_fuel_flow_kgs]

# ==============================================================================
# 3. CRIAÇÃO DO DATAFRAME
# ==============================================================================

# Estrutura os dados para o Plotly
plot_df = pd.DataFrame({
    "Tração Líquida / Tração TO [-]": lto_thrust_percentages,
    "Meta-Modelo": meta_modelo_normalized,
    "ICAO": icao_normalized,
})
logger.info(f"DataFrame para plotagem:\n{plot_df}")

# ==============================================================================
# 4. PLOTAGEM COM PLOTLY GRAPH_OBJECTS (GO)
# ==============================================================================
logger.info("Gerando gráfico...")

# Inicializa a figura
fig = go.Figure()

# --- Adiciona a Linha da ICAO (Vermelha, Sólida) ---
fig.add_trace(go.Scatter(
    x=plot_df["Tração Líquida / Tração TO [-]"],
    y=plot_df["ICAO"],
    mode='lines+markers',  # Linhas e marcadores
    name='ICAO',
    line=dict(color='red', width=2, dash='solid'),
    marker=dict(color='red', size=8, symbol='circle')
))

# --- Adiciona a Linha do Meta-Modelo (Preta, Tracejada) ---
fig.add_trace(go.Scatter(
    x=plot_df["Tração Líquida / Tração TO [-]"],
    y=plot_df["Meta-Modelo"],
    mode='lines+markers',  # Linhas e marcadores
    name='Meta-Modelo',
    line=dict(color='black', width=2, dash='dash'),
    marker=dict(color='black', size=8, symbol='circle')
))

# --- Customiza o Layout do Gráfico ---
fig.update_layout(
    title=f"Validação do Meta-Modelo vs. ICAO (Motor: {motor})",
    xaxis_title="Tração Líquida / Tração TO [-]",
    yaxis_title="Vazão de Combustível / Vazão TO [-]",
    # Define os limites dos eixos para corresponder ao seu exemplo
    xaxis=dict(range=[0, 1.1]),
    yaxis=dict(range=[0, 1.1]),
    # Posiciona a legenda no canto inferior direito
    legend=dict(
        x=0.6, y=0.1,
        bgcolor='rgba(255, 255, 255, 0.5)', # Fundo semitransparente
        bordercolor='rgba(0, 0, 0, 0.5)',
        borderwidth=1
    )
)

# ==============================================================================
# 5. SALVAR E EXIBIR
# ==============================================================================

# Salva a imagem na pasta 'images'
file_name = f"{motor}_validation_plot.png"
fig.write_image(IMAGES_DIR / file_name, width=800, height=500, scale=2)
logger.info(f"Gráfico salvo em: {IMAGES_DIR / file_name}")

# Exibe o gráfico
fig.show()

2025-11-02 21:52:48,469 - INFO - Iniciando calibração do motor para validação...
2025-11-02 21:52:48,471 - INFO - Status da Calibração: {'success': True, 'stage_sequence': ['tsfc_then_thrust'], 'message_t04': 'Solution found.', 'message_mdot': 'Solution found.', 'optimal_t04': 1269.075, 'optimal_mass_flow_rate': 104.721, 'final_thrust_kN': 50.7, 'final_tsfc': 0.01755}
2025-11-02 21:52:48,473 - src.turbofan - INFO - Calculando TSFC para visualização no intervalo T04 de 1100K a 1599K...


2025-11-02 21:52:50,080 - INFO - Gerando dados para os pontos do ciclo LTO...
2025-11-02 21:52:50,096 - INFO - DataFrame para plotagem:
   Tração Líquida / Tração TO [-]  Meta-Modelo      ICAO
0                            0.07     0.130109  0.134831
1                            0.30     0.313156  0.314607
2                            0.85     0.832341  0.820225
3                            1.00     1.000000  1.000000
2025-11-02 21:52:50,096 - INFO - Gerando gráfico...
2025-11-02 21:52:51,552 - INFO - Gráfico salvo em: images\SPEY Mk511_validation_plot.png


### Turbofan fora do ponto de projeto

Agora vamos validar como o meta-modelo atua fora do ponto de projeto, confrontando os dados da simulação com os dados reais do motor, obtidos via ICAO.

In [ ]:
turbofan = Turbofan(config_ex23)
turbofan.set_sea_level_air_flow(756)
turbofan.print_outputs()

In [ ]:
turbofan.save_design_point()
turbofan.update_from_N2(0.8)
turbofan.print_config()
turbofan.print_outputs()

In [ ]:
var_and_unit = {
    "N1": "",
    "thrust": "kN",
    "tsfc": "kg/(kN.s)",
    "prf": "",
    "prc": "",
    "pr": "",
    "hot_air_flow": "kg/s",
    "u_core": "m/s",
    "t04": "K",
    "fuel_consumption": "kg/s",
    "bpr": "",
}
for variable, unit in var_and_unit.items():
    if variable in ["prf", "bpr"]:
        param = "N1"
        range_x = [0.2, 1.0]
    else:
        param = "N2"
        range_x = [0.4, 1.0]
    df = turbofan.get_values_by_changing_param(variable, param)
    px.line(
        df,
        x=param,
        y=variable,
        title=f"{variable} vs {param}",
        labels={param: f"{param} (%)", variable: unit},
        range_x=range_x,
        width=800,
        height=400,
    ).show()
    print(round(df.loc[5, variable] / df.loc[0, variable],2))

In [ ]:
from utils.aux_tools import atmosphere
import numpy as np

In [ ]:
T,p,rho,mu = atmosphere(35e3*0.3048) # 35k feet

In [ ]:
a = np.sqrt(1.4*287*T)
V = a * 0.789
Cd = 315e-4
A = 125

In [ ]:
thrust = 1/2*rho*V**2*Cd*A

In [ ]:
thrust/2